# Poisson Image Editing
**Un projet qui s'appuit sur l'article écrit par Patrick Pérez, Michel Gangnet et Andrew Blake**

**Blondel Charlotte, Cadaux Ema, Mametjanova Aijana, Cros Marion**

## Introduction

Notre projet s’intéresse à l’édition d’images en passant par les équations de Poisson.

L’édition d’une image peut être soit globale (comme l’application de filtre ou le changement de couleur) soit locale. En ce qui concerne notre projet, nous nous focaliserons sur les modifications effectuées localement. Pour cela, nous selectionnerons manuellement les régions auxquelles nous souhaitons appliquer celles-ci. Les masques binaires nous permettront de selectionner les régions en question.

Tout au long de ce notebook, nous allons vous présenter différentes techniques d'édition. Elles iront de l’insertion d’une portion d’image à une autre (une sorte de copier-coller, de clonage) au changement de texture ou de couleur local.

Le fil rouge de notre projet est l’équation aux dérivées partielles de Poisson avec conditions aux limites de Dirichlet spécifiant le Laplacien d’une fonction inconnue sur le domaine d’intérêt. Cet outil permet des éditions et des clonages sans effet de discontinuité le long de la bordure des régions selectionnées.

Nous verrons que l'unicité de cette solution permet d'otenir des algorithmes robustes d'édition.



### Imports

In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import scipy.sparse as sp
import scipy.sparse.linalg
import scipy.io
import panel as pn
import matplotlib.pyplot as plt
from panel.pane import LaTeX
import os
import requests
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
import time
from io import BytesIO
import cv2

## Insertion

L'idée de l'insertion est de copier les gradients spatiaux ∇S de l'image source S dans l'image cible T, et non les valeurs de couleur de S. Pour réaliser un tel mélange, nous cherchons une image u solution de : $min_u \int_\Omega || \nabla u - \nabla S ||^2 $ sous la contrainte $u_{D\backslash \Omega}=T$, ce qui s'écrit :

\begin{equation*}
\min_u \int_\Omega ||\nabla u-\nabla S||^2+\iota_{K}(u)
\end{equation*}

avec K l'ensemble des images qui coïncident avec la cible à l'extérieur du masque.

### Chargement des images

In [ ]:
caselist=['Ours','Soleil','Femme piscine','Homme piscine']

In [ ]:
local=1
def chargeData(name):
    if local:
        if name=='Ours':
            target=Image.open("./data/target_ocean.jpg")
            source=Image.open("./data/source_ours.jpg")
            mask=Image.open("./data/mask_ours.png")

        if name=='Femme piscine':
            target=Image.open("./data/target_ocean.jpg")
            source=Image.open("./data/source_femme_piscine.jpg")
            mask=Image.open("./data/mask_femme_piscine.jpg")

        if name=='Homme piscine':
            target=Image.open("./data/target_ocean2.jpg")
            source=Image.open("./data/source_homme_piscine.jpg")
            mask=Image.open("./data/mask_homme_piscine.jpg")
            
        if name=='Soleil':
            target=Image.open("./data/target_sun.jpg")
            source=Image.open("./data/source_sun.jpg")
            mask=Image.open("./data/mask_sun.jpg")

        source = source.resize(np.shape(target)[:2][::-1])
        mask = mask.resize(np.shape(target)[:2][::-1])

        target = np.array(target).astype(float)
        source = np.array(source).astype(float)
        mask = np.array(mask).astype(float)/255
        
    return target,source,mask

In [ ]:
target,source,mask2=chargeData('Ours')

optionsRGB=dict(width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
optionsGray=dict(cmap='gray',width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.Image((mask2*255).astype('uint8')).opts(**optionsGray))

### Calcul des gradients

On commence par définir les fonctions de Gradient et de Divergence discrets. Le **gradient discret** d'une image est utilisé pour représenter les changements d'intensité ou de couleur dans une image. La **divergence discrète** d'une image est une opération qui prend en compte les différences entre les intensités des pixels adjacents dans une image.

In [ ]:
def GradientHor(x):
    y=x-np.roll(x,1,axis=1)
    y[:,0]=0
    return y
def GradientVer(x):
    y=x-np.roll(x,1,axis=0)
    y[0,:]=0
    return y
def DivHor(x):
    N=len(x[0])
    y=x-np.roll(x,-1,axis=1)
    y[:,0]=-x[:,1]
    y[:,N-1]=x[:,N-1]
    return y
def DivVer(x):
    N=len(x)
    y=x-np.roll(x,-1,axis=0)
    y[0,:]=-x[1,:]
    y[N-1,:]=x[N-1,:]
    return y
def Gradient(x):
    y=[]
    y.append(GradientHor(x))
    y.append(GradientVer(x))
    return y
def Div(y):
    x=DivHor(y[0])+DivVer(y[1])
    return x

### Gradient et projection

On définit une fonction `proj` qui est la projection en fonction du masque de l'image source. Les pixels de l'image source correspondants aux pixels égaux à 1 du masque sont conservés tandis que ceux n'appartenant pas à cette région sont remplacés par les pixels de l'image cible.

On définit aussi une fonction calculant le gradient de notre fonctionnelle.

In [ ]:
def Proj(im,ma,iref):
    res = im*ma + iref*(1-ma)

    return res

#Gradient de la fonctionnelle
def GradientFonc(x,y):
    g = Gradient(x)
    r0 = g[0]-y[0]
    r1 = g[1] - y[1]
    res = DivHor(r0) + DivVer(r1)

    return res

### Projection Naive

Dans cette partie, on souhaite fusionner de manière simple deux images. On choisit tout d'abord l'image source, avec l'objet que l'on veut transférer (une personne, un animal etc...), on créé un masque binaire avec en blanc l'objet que l'on veut transférer et en noir le reste. On choisit une image cible sur laquelle l'objet va être fusionné.

Tout d'abord, on effectue une projection simple de l'image source sur l'image cible grâce au masque et à la fonction `proj`. On obtient donc une projection simple.

In [ ]:
def naive(im,ma,iref):
    x  = Proj(im,ma,iref)
    return np.clip(x,0,255)

In [ ]:
# Fusion naïve
res_naive  = naive(source[:,:,0],mask2[:,:,0],target[:,:,0])

In [ ]:
optionsRGB = dict(width=400, height=400)

bw_target = hv.Image((target[:,:,0]).astype('uint8')).opts(**optionsGray)
bw_source = hv.Image(source[:,:,0].astype('uint8')).opts(**optionsGray)
proj_bw = hv.Image((res_naive).astype('uint8')).opts(**optionsGray)

target_column = pn.Column("Cible", bw_target)
source_column = pn.Column("Source", bw_source)
proj_column = pn.Column("Projection naïve", proj_bw)

pn.Row(target_column, source_column, proj_column)

Après avoir fait une projection simple, on obtient une image avec une seule couleur. Pour avoir une image en couleur, nous pouvons diviser nos images source et cible en trois canaux et appliquer `proj` sur ces différents canaux

In [ ]:
# Séparation de l'image en 3 canaux

target0=target[:,:,0]
source0=source[:,:,0]
mask20=mask2[:,:,0]

target1=target[:,:,1]
source1=source[:,:,1]
mask21=mask2[:,:,1]

target2=target[:,:,2]
source2=source[:,:,2]
mask22=mask2[:,:,2]

In [ ]:
# Fusion
projRGB = np.zeros((np.shape(target)))
projRGB[:,:,0]= Proj(source0,mask2[:,:,0],target0)
projRGB[:,:,1]= Proj(source1,mask2[:,:,0],target1)
projRGB[:,:,2]= Proj(source2,mask2[:,:,0],target2)

In [ ]:
optionsRGB = dict(width=400, height=400)

rgb_target = hv.RGB((target).astype('uint8')).opts(**optionsRGB)
rgb_source = hv.RGB(source.astype('uint8')).opts(**optionsRGB)
proj_rgb = hv.RGB((projRGB).astype('uint8')).opts(**optionsRGB)

target_column = pn.Column("Cible", rgb_target)
source_column = pn.Column("Source", rgb_source)
proj_column = pn.Column("Projection naïve", proj_rgb)

pn.Row(target_column, source_column, proj_column)

Nous pouvons voir que l'image source a bien été fusionnée avec l'image source. Cependant, le masque de l'image source ne correspondant par parfaitement aux contours de l'objet que l'on veut fusionner, nous pouvons utiliser d'autres méthodes qui peuvent corriger ce problème.

### Forward Backward Poisson

L'utilisation de l'algorithme de Forward Backward de Poisson nous permet d'ajuster l'image source pour qu'elle corresponde mieux à l'image cible tout en conservant les contraintes apportées par le masque.

`FBPoissonEditing` ajuste l'image source pour qu'elle se rapproche de l'image cible à chaque itération tout en ayant pour contrainte le masque de l'image source. Pour cela, on calcule une mise à jour de x en utilisant la formule $x-s*\nabla f(x,y)$. Ensuite, on effectue une projection de cette mise à jour sur l'image cible en utilisant `proj`. Nous pouvons aussi calculer les composantes du gradient afin de visualiser son évolution.

In [ ]:
def FBPoissonEditing(targ,sour,ma,step,Niter):

    f = []
    
    # Copie de l'image source
    x = np.copy(sour)
    
    # Calcule le gradient de l'image source
    y = Gradient(sour)

    for i in range(Niter):
        
        # Calcule valeur temporaire en utilisant le gradient
        temp = x - step * GradientFonc(x,y)

        # Projete la valeur temporaire sur l'image cible en utilisant le masque
        x = Proj(temp,ma,targ)

        # Calcule les gradients de l'image modifiée
        f1,f2 = Gradient(x - targ)*ma

        # Calcule et stocke la norme L2 des gradients
        f.append(np.linalg.norm(f1,2) + np.linalg.norm(f2,2))

    return np.clip(x,0,255), f[:10]

In [ ]:
step=1/4
niter=265

# Calcul du temps d'exécution
start_time_FB = time.time()

# FB sur les 3 canaux
res_FB0,f0=FBPoissonEditing(target0,source0,mask20,step,niter)
res_FB1,f1=FBPoissonEditing(target1,source1,mask21,step,niter)
res_FB2,f2=FBPoissonEditing(target2,source2,mask22,step,niter)

end_time_FB = time.time()

# Calculez la différence pour obtenir le temps d'exécution
execution_time_FB = end_time_FB - start_time_FB

# Création d'une image couleur à 3 canaux avec les résultats obtenus
res_FB=target.copy()
res_FB[:,:,0]=res_FB0
res_FB[:,:,1]=res_FB1
res_FB[:,:,2]=res_FB2

In [ ]:
optionsRGB = dict(width=400, height=400)

proj_rgb = hv.RGB((projRGB).astype('uint8')).opts(**optionsRGB)
fb_rgb = hv.RGB(res_FB.astype('uint8')).opts(**optionsRGB)

proj_column = pn.Column("Projection naïve", proj_rgb)
fb_column = pn.Column("Forward Backward", fb_rgb)

pn.Row(proj_column, fb_column)

In [ ]:
print('Temps d\'exécution :', execution_time_FB)

L'image obtenue après avoir effectué l'algorithme Forward Backward de Poisson contient moins de défauts. En effet, les bords autour de l'image source sont moins épais, on peut voir que cet algorithme les a mieux fusionnés avec l'image cible. De plus, l'algorithme met environ 50s à tourner pour l'oursCependant, un point faible de cet algorithme est que si les images que l'on veut traiter sont trop grandes, alors l'algorithme devient vite chronophage.

### FISTA Poisson

Au lieu d'utiliser l'algorithme de Forward-Backward, nous pouvons nous servir de FISTA.

L'algorithme `FISTAPoissonEditing` d'ajuster itérativement l'image source vers l'image cible tout en respectant les contraintes du masque. À chaque itération, on calcule une mise à jour de x en utilisant la formule $(x+\frac{n}{\alpha + n} * e) - s*\nabla f(x,y)$ avec $e$  la différence entre les itérations successives. Nous pouvons aussi calculer les composantes du gradient afin de visualiser son évolution.

In [ ]:
def FISTAPoissonEditing(targ,sour,ma,step,alpha,Niter):
    f = []
    
    # Copie de l'image source
    x = np.copy(sour)
    
    xp = np.copy(x)
    
    # Calcule le gradient de l'image source
    y = Gradient(sour)

    e = 0

    for i in range(Niter):

        # Met à jour temp avec une composante de la dernière erreur
        temp = x + Niter/(alpha+Niter) * e
        
        # Met à jour x en utilisant la méthode FISTA et la fonction de gradient
        x = temp - step * GradientFonc(temp,y)
        
        # Projet x sur l'image cible en utilisant le masque
        x = Proj(x,ma,targ)

        # Met à jour e avec la différence entre x actuel et x précédent (xp)
        e = x - xp

        # Met à jour xp
        xp = np.copy(x)

        # Calcule les gradients de l'image modifiée
        f1,f2 = Gradient(x - targ)*ma

        # Calcule et stocker la norme L2 des gradients
        f.append(np.linalg.norm(f1,2) + np.linalg.norm(f2,2))


    return np.clip(x,0,255),f[10:]

In [ ]:
step=1/10
niter=300

# Calcul du temps d'exécution
start_time_FISTA = time.time()

# FISTA sur les 3 canaux
res_FISTA0,f0=FISTAPoissonEditing(target0,source0,mask20,step,3,niter)
res_FISTA1,f1=FISTAPoissonEditing(target1,source1,mask21,step,3,niter)
res_FISTA2,f2=FISTAPoissonEditing(target2,source2,mask22,step,3,niter)

end_time_FISTA = time.time()

# Calculez la différence pour obtenir le temps d'exécution
execution_time_FISTA = end_time_FISTA - start_time_FISTA

# Création d'une image couleur à 3 canaux avec les résultats obtenus
res_FISTA=target.copy()
res_FISTA[:,:,0]=res_FISTA0
res_FISTA[:,:,1]=res_FISTA1
res_FISTA[:,:,2]=res_FISTA2

In [ ]:
optionsRGB = dict(width=400, height=400)

proj_rgb = hv.RGB((projRGB).astype('uint8')).opts(**optionsRGB)
fista_rgb = hv.RGB(res_FISTA.astype('uint8')).opts(**optionsRGB)

proj_column = pn.Column("Projection naïve", proj_rgb)
fista_column = pn.Column("FISTA", fista_rgb)

pn.Row(proj_column, fista_column)

In [ ]:
print('Temps d\'exécution :', execution_time_FISTA)

L'image obtenue après avoir effectué l'algorithme FISTA de Poisson contient, comme Forward Backward, moins de défauts. Les bords de l'image source sont toujours moins présents et l'algorithme met environ 50s à tourner pour l'ours. Cependant, cet algorithme est aussi chronophage avec de grandes images.

### Forward Backward vs FISTA

In [ ]:
optionsRGB = dict(width=400, height=400)

fb_rgb = hv.RGB((res_FB).astype('uint8')).opts(**optionsRGB)
fista_rgb = hv.RGB(res_FISTA.astype('uint8')).opts(**optionsRGB)

fb_column = pn.Column("Forward Backward", fb_rgb)
fista_column = pn.Column("FISTA", fista_rgb)

pn.Row(fb_column, fista_column)

L’algorithme FISTA fusionne davantage l’image source sur l’image cible. On voit que les couleurs de l’image cible sont plus présentes par rapport à l’algorithme Forward Backward. Au niveau du temps de calcul, les deux algorithmes ont un temps d’exécution similaire.

## Edition d'image par Poisson

La résolution de l’équation de Poisson peut être interprétée comme un problème de minimisation, calculant la fonction dont le gradient est le plus proche, en norme **L2**, d’un champ vectoriel prescrit (le champ vectoriel de guidage), sous des conditions aux limites données. Ceci permet d’interpoler les conditions aux limites vers l’intérieur, tout en suivant de près les variations spatiales du champ de guidage.

Les deux prochaines méthodes que nous vous présentons sont basées sur la résolution de l'équation de Poisson pour effectuer des modifications locales de couleur/texture à une image, en utilisant un masque préconcu par l'utilisateur.

L'équation de Poisson discrète pour une image en couleur avec trois canaux (R : Rouge, V : Vert, B : Bleu) peut être représentée par le système linéaire **Ab = u**, où :


  - **A** est la matrice de Poisson construite à partir du masque et des indices des pixels du masque.
  - **b** est le vecteur inconnu que nous voulons résoudre, représentant les changements de couleur/texture que nous appliquerons à l'image.
  - **u** est le vecteur de gradient de couleur construit à partir de l'image source, du masque et des conditions au limites (bord).


## Local Color Changes

La fonction `coefficient_matrix(id_mask, mask2id)` construit la matrice de Poisson `A` nécessaire pour résoudre le système d'équations de la méthode de Poisson.

Cette dernière est construite en suivant les règles de l'équation de Poisson discrète. Pour chaque pixel **i** appartenant au masque, l'élément diagonal de la ligne correspondante dans `A` est fixé à **4** selon le "schéma à cinq points" de la méthode des différences finies (discrétisation au deuxième ordre de l'équation de Poisson), et les éléments correspondants aux voisins dans `A` sont fixés à **-1**. Si le voisin est en dehors du masque, l'élément dans `A` reste nul pour maintenir la condition de Dirichlet.

En résumé :

- **$A[i, i]$ = 4**

- **$A[i, j]$ = -1** pour chaque voisin **j** du pixel **i** dans le masque

<img src="./notebook_image/poisson_sys.png" width="600"/>

D'autre part, la fonction `compute_u_index(source, mask, index, edge_mask)` calcule le gradient d'un pixel donné ainsi que la condition de Dirichlet sur le bord. Elle prend en compte la condition de Dirichlet (la valeur de pixels hors du masque) et les gradients des pixels voisins dans le masque.

Mathématiquement, pour chaque canal **k** (rouge, vert, bleu) et pour chaque pixel **i** appartenant au masque, le composant

**$u_i^k$** est calculé comme suit :

 - **$u_i^k$** = condition de Dirichlet pour le pixel **i** + somme des valeurs des pixels voisins appartenant au masque pour le canal **k** - somme des valeurs des pixels voisins qui ne sont pas inclus dans le masque


<img src="./notebook_image/dirichlet.png" width="400"/>

In [ ]:
def coefficient_matrix(id_mask, mask2id):
  '''
    Construit la matrice de poisson A
  '''

  ## Création d'une matrice vide
  N = id_mask.shape[0] # Nombre de points appartenant au masque
  A = sp.lil_matrix((N, N), dtype=np.float32)

  h, w = mask.shape # Taille de l'image

  for i in range(N):

    A[i, i] = 4 # Tous les éléments diagonaux de la matrice sont égaux à 4

    id_h, id_w = id_mask[i]
    neighbors = [(id_h, id_w + 1),(id_h, id_w - 1),(id_h + 1, id_w),(id_h - 1, id_w)]

    # On itère sur les pixels voisins
    for idx_h, idx_w in neighbors:

        # Si le pixel appartient au masque, alors on assigne -1 à l'endroit qui lui correspond dans la matrice
        if 0 <= idx_h < h and 0 <= idx_w < w and mask2id[idx_h][idx_w]:

            j = mask2id[idx_h][idx_w]
            A[i, j] = -1

  return A

def compute_u_index(source, mask, index):
  '''
  Calcule le gradient d'un pixel donné ainsi que la condition de dirichlet sur le bord
  '''
  i, j = index

  N = 0.0
  dircht_ix = 0.0
  grad_idx = 0.0

  neighbors = [(i, j+1), (i, j-1), (i+1, j), (i-1, j)]

  ## On itère sur les voisins du pixel considéré
  for idx in range(len(neighbors)) :

      i_ngb, j_ngb = neighbors[idx]

      # Si le voisin est hors du masque, on rajoute la valeur de son pixel (condition de dirichlet sur le bord)
      dircht_ix += float(not(mask[i_ngb, j_ngb])) * source[i_ngb, j_ngb]

      N += mask[i_ngb, j_ngb]
      grad_idx -= float(mask[i_ngb, j_ngb])*source[i_ngb, j_ngb]
  return dircht_ix + N*source[i,j] + grad_idx

def color_gradients(src, mask, id_mask, r, g, b):
  '''
    Construit le vecteur de gradient u
  '''
  N = id_mask.shape[0] # Nombre de points appartenant au masque

  # On créé un tableau par channel de couleur
  u_b = np.zeros(N)
  u_g = np.zeros(N)
  u_r = np.zeros(N)

  # Dilatation des contours obtenus
  # Plus la taille du kernel est importante, plus les contours seront larges.
  kernel = np.ones((3, 3), dtype=np.uint8)

  # On calcule le gradient de chacun des pixels appartenant au masque
  for index in range(N):

    i, j = id_mask[index]
    #on multiplie chaque canal de couleur de b par b,g,r afin de modifier en local la couleur de l'image
    u_b[index] = compute_u_index(src[:, :, 0], mask, id_mask[index])*b
    u_g[index] = compute_u_index(src[:, :, 1], mask, id_mask[index])*g
    u_r[index] = compute_u_index(src[:, :, 2], mask, id_mask[index])*r

  return u_b, u_g, u_r

La fonction `color_gradients(src, mask, id_mask, r, g, b)` construit le vecteur de gradient **b** pour chaque canal de couleur (rouge, vert, bleu) en utilisant la fonction `compute_u_index` pour chaque pixel appartenant au masque. De plus, afin de modifier la couleur, on multiplie chaque canal de couleur par des coefficients spécifiques.
Par exemple, pour l'image d'un pissenlit jaune, multiplier le canal rouge par **1.5** augmente l'intensité du rouge, tandis que multiplier les canaux vert et bleu par **0.5** réduit leur intensité. Cela a pour effet global de déplacer la couleur vers le rouge-orange.


La fonction `poisson_local_color_change(src, mask)` applique la modification souhaitée au regard des paramètres définis plus haut.

Dans un premier temps, on seuille le masque pour obtenir un masque binaire.
On récupère ensuite les coordonnées des éléments du masque et les indices correspondants.
On construit la matrice **A** et le vecteur afin de résoudre le système d'équations linéaires **Au = b** pour chaque canal de couleur.
Enfin, on remplace les valeurs des pixels appartenant au masque dans l'image source par les valeurs obtenus de **u**.

In [ ]:
def poisson_local_color_change(src, mask, r, g, b):
    # Seuillage du masque
    _, mask_bin = cv2.threshold(mask, 0, 255, cv2.THRESH_OTSU)
    mask_bin = mask_bin/255

    # id_mask est un tableau contenant les coordonnées (x, y) des éléments appartenant au masque
    id_mask = np.argwhere(mask_bin)

    # mask2id est un tableau de la même taille que la source.
    # Chaque pixel appartenant au masque est égal à la valeur de son index dans id_mask, les autres pixels sont nuls
    mask2id = np.zeros(mask.shape, dtype=np.int32)

    for index, (i, j) in enumerate(id_mask):
        mask2id[i][j] = index

    ###

    # Remplit la matrice A
    print("Step 1: Filling coefficient matrix A")
    A = coefficient_matrix(id_mask, mask2id)
    print("\n")

    # Remplit la matrice u
    print("Step 2: Filling gradient matrix u")
    u_b,u_g,u_r= color_gradients(src, mask_bin, id_mask, r, g, b)
    print("\n")

    # Résout le système pour chaque canal de couleur
    print("Step 3: Solve Au = b")
    x_b, _ = sp.linalg.cg(A, u_b)
    x_g, _ = sp.linalg.cg(A, u_g)
    x_r, _ = sp.linalg.cg(A, u_r)
    print("done!\n")

    # On remplace dans la source la valeur des pixels appartenant au masque par les valeurs calculées
    erased = src.copy()
    for index, (i, j) in enumerate(id_mask):

        erased[i][j][0] = np.clip(x_b[index], 0.0, 1.0)
        erased[i][j][1] = np.clip(x_g[index], 0.0, 1.0)
        erased[i][j][2] = np.clip(x_r[index], 0.0, 1.0)

    return np.array(erased*255, dtype=np.uint8)

In [ ]:
from google.colab.patches import cv2_imshow

**Etude de cas 1 : Pissenlit**

In [ ]:
# Importation de la source et du masque
src_path = './data/image_pissenlit.jpg'
src_mask_path = './data/image_pissenlit_masque.jpg'

filename_src, ext_src = os.path.splitext( os.path.basename(src_path) )
output_name = './data/' + filename_src + '_processed' + ext_src

src = np.array(cv2.imread(src_path, 1)/255.0, dtype=np.float32)
mask = np.array(cv2.imread(src_mask_path, 0), dtype=np.uint8)

# Réalisation du changement de couleur ~ 5 min
erased = poisson_local_color_change(src,mask, 1.5, 0.5, 0.5)

# Sauvegarde des résultats dans ./data/
merged_result = np.hstack((np.array(src*255, dtype=np.uint8), cv2.merge((mask, mask, mask)), erased))
cv2.imwrite(output_name, merged_result)
cv2_imshow(merged_result)

**Etude de cas 2 : Poivron**

In [ ]:
# Importation de la source et du masque
src_path = './data/poivron.png'
src_mask_path = './data/masque_poivron_2.png'

filename_src, ext_src = os.path.splitext( os.path.basename(src_path) )
output_name = './data/' + filename_src + '_processed' + ext_src

src = np.array(cv2.imread(src_path, 1)/255.0, dtype=np.float32)
mask = np.array(cv2.imread(src_mask_path, 0), dtype=np.uint8)

# Réalisation du changement de couleur ~ 11 minutes
erased = poisson_local_color_change(src,mask, 2, 0.25, 0.25)

# Sauvegarde des résultats dans ./data/
merged_result = np.hstack((np.array(src*255, dtype=np.uint8), cv2.merge((mask, mask, mask)), erased))
cv2.imwrite(output_name, merged_result)
cv2_imshow(merged_result)

**Etude de cas 3: Chat**

In [ ]:
# Importation de la source et du masque
src_path = './data/chat_orange.jpg'
src_mask_path = './data/chat_orange_masque.jpg'

filename_src, ext_src = os.path.splitext( os.path.basename(src_path) )
output_name = './data/' + filename_src + '_processed' + ext_src

src = np.array(cv2.imread(src_path, 1)/255.0, dtype=np.float32)
mask = np.array(cv2.imread(src_mask_path, 0), dtype=np.uint8)

# Réalisation du changement de couleur ~
erased = poisson_local_color_change(src,mask, 1.6, 0.8, 0.9)

# Sauvegarde des résultats dans ./data/
merged_result = np.hstack((np.array(src*255, dtype=np.uint8), cv2.merge((mask, mask, mask)), erased))
cv2.imwrite(output_name, merged_result)
cv2_imshow(merged_result)

## Face fattening

L'anonymisation des visages est primordiale dans divers contextes pour protéger la vie privée des individus et garantir le respect des règles de confidentialité :

- En sciences sociales et en recherche clinique par exemple, de nombreuses études impliquent des participants humains. L'anonymisation des visages garantit que ceux-ci ne soient pas identifiables dans les données collectées. Il est est de même dans le domaine de la médecine pour l'anonymisation des patients.

- L'entraînement des algorithmes de reconnaissance faciale est réalisé grâce à des bases de données nécessitant l'anonymisation des individus afin d'assurer le respect de la confidentialité.

- Lors de la diffusion d'images provenant de caméras de surveillance publiques, l'anonymisation des visages contribue à protéger la vie privée des citoyens tout en permettant d’assurer la surveillance nécessaire. Une mauvaise utilisation des bases de données peut créer des polémiques comme en 2021 lors de l'utilisation de ClearView AI par la police pour traquer les criminels. Les milliards de visages présents dans la base de données ont soulevé des préoccupations dans la population sur le droit de la police d'utiliser une telle base de données à des fins de surveillance criminelle.

- Les médias utilisent également des techniques d'anonymisation des visages afin de protéger l'identité des personnes impliquées dans des reportages ou des documentaires sensibles.


Ainsi, l’utilisation de telles bases de données pose de nombreux problèmes éthiques, juridiques et scientifiques. Les risques pour la vie privée sont encore plus importants si l'on considère les conséquences de l’utilisation de ces données à des fins détournées de leur but initial.

Pour empêcher l'utilisation abusive des ensembles de données tout en permettant le développement d'applications futures, il est donc primordial que les ensembles de données puissent être anonymisés de manière à préserver l'utilité des données.

Les images faciales représentent l'un des types d'informations les plus complexes car les visages fournissent une représentation directe de l'identité des êtres humains. Pour préserver les caractéristiques faciales des données, la possibilité étudiée dans ce Notebook est de préserver les yeux, les lèvres et le nez de l’image originale.

La démarche utilisée reprend celle décrite au-dessus pour le changement de couleur local appliqué à une région donnée. Seul le calcul du vecteur de gradient u diffère.

Le vecteur **u** est obtenu en additionnant le gradient d’un pixel donné avec la condition de Dirichlet sur le bord du masque. 
La condition de Dirichlet d’un pixel du masque est obtenue additionnant les valeurs de ses pixels voisins si ceux-ci n'appartiennent pas au masque.

<img src="./notebook_image/gradient.png" width="300"/>

In [ ]:
def coefficient_matrix(id_mask, mask2id):
  '''
    Construit la matrice de poisson A
  '''

  ## Création d'une matrice vide
  N = id_mask.shape[0] # Nombre de points appartenant au masque
  A = sp.lil_matrix((N, N), dtype=np.float32)

  h, w = mask.shape # Taille de l'image

  for i in range(N):

    A[i, i] = 4 # Tous les éléments diagonaux de la matrice sont égaux à 4

    id_h, id_w = id_mask[i]
    neighbors = [(id_h, id_w + 1),(id_h, id_w - 1),(id_h + 1, id_w),(id_h - 1, id_w)]

    # On itère sur les pixels voisins
    for idx_h, idx_w in neighbors:

        # Si le pixel appartient au masque, alors on assigne -1 à l'endroit qui lui correspond dans la matrice
        if 0 <= idx_h < h and 0 <= idx_w < w and mask2id[idx_h][idx_w]:

            j = mask2id[idx_h][idx_w]
            A[i, j] = -1

  return A

def compute_u_ij(source, mask, index, edge_mask):
  '''
  Calcule le gradient d'un pixel donné ainsi que la condition de dirichlet sur le bord
  '''
  i, j = index

  dircht_ix = 0.0
  grad_idx = 0.0

  neighbors = [(i, j+1), (i, j-1), (i+1, j), (i-1, j)]
  cor_edge = [(i, j), (i, j-1), (i, j), (i-1, j)]

  ## On itère sur les voisins du pixel considéré
  for idx in range(len(neighbors)) :

      i_ngb, j_ngb = neighbors[idx]
      i_edg, j_edg = cor_edge[idx]

      # Si le voisin est hors du masque, on rajoute la valeur de son pixel (condition de dirichlet sur le bord)
      dircht_ix += float(not(mask[i_ngb, j_ngb])) * source[i_ngb, j_ngb]

      # Si le voisin appartient au masqu, on calcule son gradient
      grad_idx += float(mask[i_ngb, j_ngb]) * (source[i, j]-source[i_ngb, j_ngb]) * edge_mask[i_edg][j_edg]

  return dircht_ix + grad_idx

def texture_flatten(src, mask, id_mask):
  '''
    Construit le vecteur de gradient u
  '''
  N = id_mask.shape[0] # Nombre de points appartenant au masque

  # On créé un tableau par channel de couleur
  u_b = np.zeros(N)
  u_g = np.zeros(N)
  u_r = np.zeros(N)

  ### Extraction des countours de l'image source (ex. yeux, cheveux,..)
  gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
  raw_edge = cv2.Canny(np.array(gray*255, dtype=np.uint8), 100, 200)

  # Dilatation des contours obtenus
  # Plus la taille du kernel est importante, plus les contours seront larges.
  kernel = np.ones((3, 3), dtype=np.uint8)
  edge_mask = cv2.dilate(raw_edge, kernel, iterations=1)/255

  # On calcule le gradient de chacun des pixels appartenant au masque
  for index in range(N):

    i, j = id_mask[index]

    u_b[index] = compute_u_ij(src[:, :, 0], mask, id_mask[index], edge_mask)
    u_g[index] = compute_u_ij(src[:, :, 1], mask, id_mask[index], edge_mask)
    u_r[index] = compute_u_ij(src[:, :, 2], mask, id_mask[index], edge_mask)

  return edge_mask, u_b, u_g, u_r

In [ ]:
def poisson_face_flatten(src, mask):
    # Seuillage du masque avec la méthode Otsu
    _, mask_bin = cv2.threshold(mask, 0, 255, cv2.THRESH_OTSU)
    mask_bin = mask_bin/255

    # id_mask est un tableau contenant les coordonnées (x, y) des éléments appartenant au masque
    id_mask = np.argwhere(mask_bin)

    # mask2id est un tableau de la même taille que la source.
    # Chaque pixel appartenant au masque est égal à la valeur de son index dans id_mask, les autres pixels sont nuls
    mask2id = np.zeros(mask.shape, dtype=np.int32)

    for index, (i, j) in enumerate(id_mask):
        mask2id[i][j] = index

    ###

    # Remplit la matrice A
    print("Step 1: Filling coefficient matrix A")
    A = coefficient_matrix(id_mask, mask2id)
    print("\n")

    # Remplit la matrice u
    print("Step 2: Filling gradient matrix u")
    edge_mask,u_b,u_g,u_r= texture_flatten(src, mask_bin, id_mask)
    print("\n")

    # Résout le système pour chaque canal de couleur
    print("Step 3: Solve Au = b")
    x_b, _ = sp.linalg.cg(A, u_b)
    x_g, _ = sp.linalg.cg(A, u_g)
    x_r, _ = sp.linalg.cg(A, u_r)
    print("done!\n")

    # On remplace dans la source la valeur des pixels appartenant au masque par les valeurs calculées
    erased = src.copy()
    for index, (i, j) in enumerate(id_mask):

        erased[i][j][0] = np.clip(x_b[index], 0.0, 1.0)
        erased[i][j][1] = np.clip(x_g[index], 0.0, 1.0)
        erased[i][j][2] = np.clip(x_r[index], 0.0, 1.0)

    return edge_mask,np.array(erased*255, dtype=np.uint8)


In [ ]:
# Importation de la source et du masque
src_path = './data/source_awkward.png'
src_mask_path = './data/mask_awkward.png'

filename_src, ext_src = os.path.splitext( os.path.basename(src_path) )
output_name = './data/' + filename_src + '_processed' + ext_src

src = np.array(cv2.imread(src_path, 1)/255.0, dtype=np.float32)
mask = np.array(cv2.imread(src_mask_path, 0), dtype=np.uint8)

# Réalisation du floutage, temps de compilation ~ 2 minutes
edge_mask,erased = poisson_face_flatten(src,mask)

# Sauvegarde des résultats dans ./data/
merged_result = np.hstack((np.array(src*255, dtype=np.uint8), cv2.merge((mask, mask, mask)), cv2.merge((edge_mask*255, edge_mask*255, edge_mask*255)), erased))
cv2.imwrite(output_name, merged_result)

**Tests effectués**

<img src="./data/source_lesggy_processed.jpg" width="800"/>
<img src="./data/source_ajoli_processed.jpg" width="800"/>
<img src="./data/source_chalamet_processed.png" width="800"/>
<img src="./data/source_awkward_processed.png" width="800"/>
<img src="./data/source_killian_processed.jpg" width="800"/>

L'algorithme offre de bonnes performances, il fonctionne même sur des personnes portant des lunettes. Cependant, la barbe, la texture de peau ainsi que les rides perturbent le floutage. 

## Mixed seamless cloning

### Charging data 

In [ ]:
caselist=['Function','Cheese','Donut', 'Rainbow']

In [ ]:
def charge_data_MPE(case:str):
    if case==caselist[0]:
        target=np.array(Image.open("./data/function_target.png")).astype(float)
        source=np.array(Image.open("./data/function_source.png")).astype(float)
        mask=np.array(Image.open("./data/function_mask2.png")).astype(float)/255
        mask=mask[:,:,0]
        mask_elab=np.array(Image.open("./data/function_mask.png")).astype(float)/255 #masque élaborée qui répète exactement la forme de l'objet à insérer
    elif case==caselist[1]:
        target=np.array(Image.open("./data/cheese_target.png")).astype(float)
        source=np.array(Image.open("./data/cheese_source.png")).astype(float)
        mask=np.array(Image.open("./data/cheese_mask.png")).astype(float)/255
        mask=mask[:,:,0]
        mask_elab=None
    elif case==caselist[2]:
        target=np.array(Image.open("./data/donut_target.jpeg")).astype(float)
        source=np.array(Image.open("./data/donut_source.jpeg")).astype(float)
        mask=np.array(Image.open("./data/donut_mask.jpeg")).astype(float)/255
        mask_elab=None
    else:
        target=np.array(Image.open("./data/rainbow_target.png")).astype(float)
        source=np.array(Image.open("./data/rainbow_source.png")).astype(float)
        mask=np.array(Image.open("./data/rainbow_mask.png")).astype(float)/255
        mask=mask[:,:,0]
        mask_elab=None

    return target, source,mask,mask_elab

### First case: example from the original paper

In [ ]:
target,source,mask,mask_elab=charge_data_MPE("Function")

In [ ]:
optionsRGB=dict(width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
optionsGray=dict(cmap='gray',width=300,height=300,xaxis=None,yaxis=None,toolbar=None)

pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB))

Afin de réaliser une insertion de l'image source (à gauche) dans l'image target (à droite), nous avons besoin de créer un masque. Nous pouvons créer un masque élaboré qui prend en compte la forme de l'image source avec les trous. Cependant, nous verrons par la suite qu'il existe une technique de Poisson Image Editing appelée mixed seamless cloning qui nous permettra d'éviter ce tracas et d'utiliser un masque très simple, sans sélection élaborée. Nous utiliserons donc par la suite le masque à gauche.

In [ ]:
pn.Row(hv.Image((mask*255).astype('uint8')).opts(**optionsGray),hv.Image((mask_elab*255).astype('uint8')).opts(**optionsGray))

In [ ]:
def Proj(im,ma,iref): 
    tmp=iref*(np.ones(np.shape(ma))-ma)
    tmp2=im*ma
    res=tmp+tmp2
    return res

def GradientFonc(x,y):
    res=[]
    res.append(Gradient(x)[0]-y[0])
    res.append(Gradient(x)[1]-y[1])
    res1=Div(res)
    return res1

Afin d'appliquer la méthode de mixed seamless cloning, nous allons modifier guidance field en appliquant la formule donnée dans l'article :

$$
v(x)=
\begin{cases}
    \nabla f^*(x) & \text{si } |\nabla f^*(x)| > |\nabla g (x)|, \\
    \nabla g(x) & \text{sinon},
\end{cases}
$$
ou bien sa version  discrétisée : 
$$
v_{pq} =
\begin{cases}
    f_p^* - f_q^* & \text{si } |f_p^* - f_q^*| > |g_p - g_q|, \\
    g_p - g_q & \text{sinon},
\end{cases}
$$
pour tous les $<p,q>$. 

On rappelle que $<p,q>$ est un couple de pixels avec $p \in S $ ($S$ = image source), $q \in N_p (N_p$ = ensemble des quatre voisins de $p$), $f^*$ est le gradient de l'image cible et $g$ est le gradient de l'image source.  

In [ ]:
def guidance_field(f, g):
    grad_f = Gradient(f)
    grad_g = Gradient(g)
    grad = []
    for i in range(2):
        mask  = np.abs(grad_f[i]) > np.abs(grad_g[i])
        grad_elem = np.where(mask, grad_f[i], grad_g[i])
        grad.append(grad_elem)
    return grad

Nous séparons les images sources et target par chaque canal.

In [ ]:
target0=target[:,:,0]
source0=source[:,:,0]
target1=target[:,:,1]
source1=source[:,:,1]
target2=target[:,:,2]
source2=source[:,:,2]

In [ ]:
def FBMixedPoissonEditing(targ,sour,ma,step,Niter):
    x=sour
    f=[]
    guidance = guidance_field(targ, sour)
    for i in range(Niter):
        x=Proj(x-step*GradientFonc(x,guidance),ma,targ)
        aux=[]
        aux.append(Gradient(x)[0]-guidance[0])
        aux.append(Gradient(x)[1]-guidance[1])
        F_u=1/2*np.linalg.norm(aux)**2
        f.append(F_u)
    return np.clip(x,0,255),f[10:]

In [ ]:
def FISTAMixedPoissonEditing(targ,sour,ma,step,alpha,Niter):
    x=sour.copy()
    f=[]
    x_old=x.copy()
    alpha_old = alpha
    guidance = guidance_field(targ, sour)
    for i in range(Niter):
        alpha_new = (1 + np.sqrt(1 + 4 * alpha_old**2)) / 2
        y = x + ((alpha_old - 1) / alpha_new) * (x - x_old)
        x_old=x.copy()
        x=Proj(y-step*GradientFonc(y,guidance),ma,targ)
        aux=[]
        aux.append(Gradient(x)[0]-guidance[0])
        aux.append(Gradient(x)[1]-guidance[1])
        F_u=1/2*np.linalg.norm(aux)**2
        f.append(F_u)
        alpha_old = alpha_new
    return np.clip(x,0,255),f[10:]

Nous utilisons ici l'algorithme FISTA telle que codé en TP avec une modification de alpha. Si en TP nous avons utilisé un alpha constant (y était alors égal à $x + \alpha * (x-x_{old})$), ici cela ne nous donnait pas de résultats satisfaisants. Nous avons donc utilisée une formule de mise à jour de alpha et nous verrons par la suite que nous obtenons des résultats satisfaisants. 

In [ ]:
Niter=1000

In [ ]:
res0_FB,f_FB=FBMixedPoissonEditing(target0,source0,mask,1/8,Niter)
res1_FB,f_FB=FBMixedPoissonEditing(target1,source1,mask,1/8,Niter)
res2_FB,f_FB=FBMixedPoissonEditing(target2,source2,mask,1/8,Niter)

In [ ]:
res0_FISTA,f_FISTA=FISTAMixedPoissonEditing(target0,source0,mask,1/8,1,Niter)
res1_FISTA,f_FISTA=FISTAMixedPoissonEditing(target1,source1,mask,1/8,1,Niter)
res2_FISTA,f_FISTA=FISTAMixedPoissonEditing(target2,source2,mask,1/8,1,Niter)

In [ ]:
plt.plot(np.linspace(10,Niter,Niter-10), f_FB, label='Cost function FB', color='blue') 
plt.plot(np.linspace(10,Niter,Niter-10), f_FISTA, label='Cost function FISTA', color='red')  

plt.xlabel('iterations')
plt.ylabel('f')
plt.title('Cost function: FB vs FISTA')
plt.legend()

plt.show()

In [ ]:
res_f=np.zeros((95,104,3))
res_f[:,:,0]=res0_FB
res_f[:,:,1]=res1_FB
res_f[:,:,2]=res2_FB
pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))


In [ ]:
res_f=np.zeros((95,104,3))
res_f[:,:,0]=res0_FISTA
res_f[:,:,1]=res1_FISTA
res_f[:,:,2]=res2_FISTA
pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))



Nous observons que nous obtenons des résultats satisfaisants pour 1000 itérations avec la téchnique de Mixed Poisson Editing. De plus, nous avons constaté que dans ce cas, un alpha constant dans l'algorithme FISTA ne donnait pas des résultats satisfaisants. Nous avons donc implementé la formule de mise à jour de alpha issue des méthodes de gradient accélérées de Yurii Nesterov.
Comparons ces resultats avec les images obtenues en utilisant la téchnique de seamless cloning où l'on l'utilise le gradient de l'image source comme guidance field.

In [ ]:
def FBPoissonEditing_seamless_cloning(targ,sour,ma,step,Niter):
    x=sour
    f=[]
    for i in range(Niter):
        x=Proj(x-step*GradientFonc(x,Gradient(sour)),ma,targ)
        aux=[]
        aux.append(Gradient(x)[0]-Gradient(sour)[0])
        aux.append(Gradient(x)[1]-Gradient(sour)[1])
        F_u=1/2*np.linalg.norm(aux)**2
        f.append(F_u)
        #f=GradientFonc
    return np.clip(x,0,255),f[10:]

In [ ]:
res0_SC,f_SC=FBPoissonEditing_seamless_cloning(target0,source0,mask,1/8,Niter)
res1_SC,f_SC=FBPoissonEditing_seamless_cloning(target1,source1,mask,1/8,Niter)
res2_SC,f_SC=FBPoissonEditing_seamless_cloning(target2,source2,mask,1/8,Niter)
res_f=np.zeros((95,104,3))
res_f[:,:,0]=res0_SC
res_f[:,:,1]=res1_SC
res_f[:,:,2]=res2_SC

In [ ]:
pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))


In [ ]:
def FISTAPoissonEditing_seamless_cloning(targ,sour,ma,step,alpha,Niter):
    x=sour
    f=[]
    x_old=x.copy()
    alpha_old=alpha
    for i in range(Niter):
        alpha_new = (1 + np.sqrt(1 + 4 * alpha_old**2)) / 2
        y = x + ((alpha_old - 1) / alpha_new) * (x - x_old)
        x_old=x.copy()
        x=Proj(y-step*GradientFonc(y,Gradient(sour)),ma,targ)
        
        aux=[]
        aux.append(Gradient(x)[0]-Gradient(sour)[0])
        aux.append(Gradient(x)[1]-Gradient(sour)[1])
        F_u=1/2*np.linalg.norm(aux)**2
        f.append(F_u)
        alpha_old=alpha_new
    return np.clip(x,0,255),f[10:]

In [ ]:
res0_FISTA,f_FISTA=FISTAPoissonEditing_seamless_cloning(target0,source0,mask,1/8,1,Niter)
res1_FISTA,f_FISTA=FISTAPoissonEditing_seamless_cloning(target1,source1,mask,1/8,1,Niter)
res2_FISTA,f_FISTA=FISTAPoissonEditing_seamless_cloning(target2,source2,mask,1/8,1,Niter)
res_f=np.zeros((95,104,3))
res_f[:,:,0]=res0_FISTA
res_f[:,:,1]=res1_FISTA
res_f[:,:,2]=res2_FISTA

In [ ]:
pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))


In [ ]:
plt.plot(np.linspace(10,Niter,Niter-10), f_SC, label='Cost function FB', color='blue') 
plt.plot(np.linspace(10,Niter,Niter-10), f_FISTA, label='Cost function FISTA', color='red')  

plt.xlabel('iterations')
plt.ylabel('f')
plt.title('Cost function: FB vs FISTA')
plt.legend()

plt.show()

Nous observons que nous obtenons des résultats beaucoup moins satisfaisants avec la téchnique de seamless cloning. En effet, cette téchnique ne prend pas en compte la forme spécifique de l'image source, notamment les trous. 

Nous constatons alors qu'avec une simple modification de guidance field, Mixed Poisson Editing permet d'obtenir de bons resultats. Il est alors crucial d'adapter le guidance field au problème de Image Editing donné. 

### Second case: cheddar cheese

In [ ]:
target,source,mask,mask_elab=charge_data_MPE("Cheese")

In [ ]:
target0=target[:,:,0]
source0=source[:,:,0]
target1=target[:,:,1]
source1=source[:,:,1]
target2=target[:,:,2]
source2=source[:,:,2]

In [ ]:
optionsRGB=dict(width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
optionsGray=dict(cmap='gray',width=300,height=300,xaxis=None,yaxis=None,toolbar=None)

pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.Image((mask*255).astype('uint8')).opts(**optionsGray))

In [ ]:
Niter=1000
res0_FB,f_FB=FBMixedPoissonEditing(target0,source0,mask,1/8,Niter)


In [ ]:
res1_FB,f_FB=FBMixedPoissonEditing(target1,source1,mask,1/8,Niter)


In [ ]:
res2_FB,f_FB=FBMixedPoissonEditing(target2,source2,mask,1/8,Niter)

In [ ]:
h,w,c=np.shape(target)
res_f=np.zeros((h,w,c))
res_f[:,:,0]=res0_FB
res_f[:,:,1]=res1_FB
res_f[:,:,2]=res2_FB
pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))


In [ ]:
res0_FISTA,f_FISTA=FISTAMixedPoissonEditing(target0,source0,mask,1/8,1,Niter)
res1_FISTA,f_FISTA=FISTAMixedPoissonEditing(target1,source1,mask,1/8,1,Niter)
res2_FISTA,f_FISTA=FISTAMixedPoissonEditing(target2,source2,mask,1/8,1,Niter)

In [ ]:
h,w,c=np.shape(target)
res_f=np.zeros((h,w,c))
res_f[:,:,0]=res0_FISTA
res_f[:,:,1]=res1_FISTA
res_f[:,:,2]=res2_FISTA
pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))


In [ ]:
plt.plot(np.linspace(10,Niter,Niter-10), f_FB, label='Cost function FB', color='blue') 
plt.plot(np.linspace(10,Niter,Niter-10), f_FISTA, label='Cost function FISTA', color='red')  

plt.xlabel('iterations')
plt.ylabel('f')
plt.title('Cost function: FB vs FISTA')
plt.legend()

plt.show()



### Third case: donut

In [ ]:
target,source,mask,mask_elab=charge_data_MPE("Donut")

In [ ]:
target0=target[:,:,0]
source0=source[:,:,0]
target1=target[:,:,1]
source1=source[:,:,1]
target2=target[:,:,2]
source2=source[:,:,2]

In [ ]:
optionsRGB=dict(width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
optionsGray=dict(cmap='gray',width=300,height=300,xaxis=None,yaxis=None,toolbar=None)

pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.Image((mask*255).astype('uint8')).opts(**optionsGray))

In [ ]:
Niter=1000
res0_FB,f_FB=FBMixedPoissonEditing(target0,source0,mask,1/8,Niter)
res1_FB,f_FB=FBMixedPoissonEditing(target1,source1,mask,1/8,Niter)
res2_FB,f_FB=FBMixedPoissonEditing(target2,source2,mask,1/8,Niter)


In [ ]:
h,w,c=np.shape(target)
res_f=np.zeros((h,w,c))
res_f[:,:,0]=res0_FB
res_f[:,:,1]=res1_FB
res_f[:,:,2]=res2_FB
pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))


In [ ]:
res0_FISTA,f_FISTA=FISTAMixedPoissonEditing(target0,source0,mask,1/8,1,Niter)
res1_FISTA,f_FISTA=FISTAMixedPoissonEditing(target1,source1,mask,1/8,1,Niter)
res2_FISTA,f_FISTA=FISTAMixedPoissonEditing(target2,source2,mask,1/8,1,Niter)

In [ ]:
h,w,c=np.shape(target)
res_f=np.zeros((h,w,c))
res_f[:,:,0]=res0_FISTA
res_f[:,:,1]=res1_FISTA
res_f[:,:,2]=res2_FISTA
pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))


In [ ]:
plt.plot(np.linspace(10,Niter,Niter-10), f_FB, label='Cost function FB', color='blue') 
plt.plot(np.linspace(10,Niter,Niter-10), f_FISTA, label='Cost function FISTA', color='red')  

plt.xlabel('iterations')
plt.ylabel('f')
plt.title('Cost function: FB vs FISTA')
plt.legend()

plt.show()



On observe que l'on obtient des résultats beaucoup moins satisfaisants dans les deux derniers cas, même avec le nombre d'itérations important. Afin d'améliorer ces résultats, nous pourrions modifier les paramètres de step et alpha ainsi que d'implémenter des masques un peu plus élaborés que ceux utilisés. 

### Fourth case: rainbow

In [ ]:
target,source,mask,mask_elab=charge_data_MPE("Rainbow")

In [ ]:
target0=target[:,:,0]
source0=source[:,:,0]
target1=target[:,:,1]
source1=source[:,:,1]
target2=target[:,:,2]
source2=source[:,:,2]

In [ ]:
optionsRGB=dict(width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
optionsGray=dict(cmap='gray',width=300,height=300,xaxis=None,yaxis=None,toolbar=None)

pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.Image((mask*255).astype('uint8')).opts(**optionsGray))




In [ ]:
Niter=1000
res0_FB,f_FB=FBMixedPoissonEditing(target0,source0,mask,1/8,Niter)
res1_FB,f_FB=FBMixedPoissonEditing(target1,source1,mask,1/8,Niter)
res2_FB,f_FB=FBMixedPoissonEditing(target2,source2,mask,1/8,Niter)

In [ ]:
h,w,c=np.shape(target)
res_f=np.zeros((h,w,3))
res_f[:,:,0]=res0_FB
res_f[:,:,1]=res1_FB
res_f[:,:,2]=res2_FB
pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))


In [ ]:
res0_FISTA,f_FISTA=FISTAMixedPoissonEditing(target0,source0,mask,1/8,1,Niter)
res1_FISTA,f_FISTA=FISTAMixedPoissonEditing(target1,source1,mask,1/8,1,Niter)
res2_FISTA,f_FISTA=FISTAMixedPoissonEditing(target2,source2,mask,1/8,1,Niter)

In [ ]:
h,w,c=np.shape(target)
res_f=np.zeros((h,w,3))
res_f[:,:,0]=res0_FISTA
res_f[:,:,1]=res1_FISTA
res_f[:,:,2]=res2_FISTA
pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))


In [ ]:
plt.plot(np.linspace(10,Niter,Niter-10), f_FB, label='Cost function FB', color='blue') 
plt.plot(np.linspace(10,Niter,Niter-10), f_FISTA, label='Cost function FISTA', color='red')  

plt.xlabel('iterations')
plt.ylabel('f')
plt.title('Cost function: FB vs FISTA')
plt.legend()

plt.show()



### Custom modifications

In [ ]:
class FISTAFusion(param.Parameterized):
    case = param.ObjectSelector(default='Function',objects=caselist)
    Niter = param.Integer(100,bounds=(10,3000))
    step = param.Number(1/8,bounds=(0.1,4))
    alpha = param.Number(0.5,bounds=(0.1,5))
    def view(self):
        target,source,ma,ma_elab=charge_data_MPE(self.case)
        target0=target[:,:,0]
        source0=source[:,:,0]
        target1=target[:,:,1]
        source1=source[:,:,1]
        target2=target[:,:,2]
        source2=source[:,:,2]
        res0,F0=FISTAMixedPoissonEditing(target0,source0,ma,self.step,self.alpha,self.Niter)
        res1,F1=FISTAMixedPoissonEditing(target1,source1,ma,self.step,self.alpha,self.Niter)
        res2,F2=FISTAMixedPoissonEditing(target2,source2,ma,self.step,self.alpha,self.Niter)
        h,w=np.shape(target)[0],np.shape(target)[1]
        res_f=np.zeros((h,w,3))
        res_f[:,:,0]=res0
        res_f[:,:,1]=res1
        res_f[:,:,2]=res2
        F=[sum(f)/3 for f in zip(F0, F1, F2)]
        F=plt.plot(F)
        return pn.Row(hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(res_f.astype('uint8')).opts(**optionsRGB))

In [ ]:
fistafusion=FISTAFusion()
pn.Row(fistafusion.param,fistafusion.view)